<a href="https://colab.research.google.com/github/LuisPalominoR/Clase-5/blob/main/Edificaciones_PROTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Cleaning in Python

The data we have collected may have several issues we need to identify:

* Are there missing values? How are they represented?
* Is the format of the table ready to be analyzed? Are there other elements not relevant but distracting or likely to confuse our work?
* Is every cell well written? are there characters that may not allow future analysis?

Let's check some data:

In [1]:
import os

In [2]:
pip show openpyxl

Name: openpyxl
Version: 3.1.2
Summary: A Python library to read/write Excel 2010 xlsx/xlsm files
Home-page: https://openpyxl.readthedocs.io
Author: See AUTHORS
Author-email: charlie.clark@clark-consulting.eu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: et-xmlfile
Required-by: 


In [3]:
import pandas as pd

In [4]:
folder="data"
fileName="Edificaciones.xlsx"
fileToRead=os.path.join(folder,fileName)

In [5]:
fileName="Edificaciones.xlsx"
fileToRead=os.path.join(folder,fileName)

dataExcel=pd.read_excel(fileToRead)

In [6]:
df_ordenado = dataExcel.sort_values(by='CODIGO_\nENTIDAD', ascending=True)

In [7]:
df_ordenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CODIGO_
ENTIDAD           180 non-null    int64 
 1   CODIGO_
UBIGEO            180 non-null    int64 
 2   CODIGO_
PAIS              180 non-null    object
 3   DEPARTAMENTO              180 non-null    object
 4   PROVINCIA                 180 non-null    object
 5   DISTRITO                  180 non-null    object
 6   NOMBRE_UO                 180 non-null    object
 7   GOBIERNO_LOCAL            180 non-null    object
 8   RUC_GOBIERNO
_LOCAL       180 non-null    int64 
 9   NUMERO_
EXPEDIENTE        180 non-null    object
 10  FECHA_
INICIO_
TRAMITE    86 non-null     object
 11  FECHA_
EMISION_TRAMITE    168 non-null    object
 12  NRO_RESOLUCION_
LICENCIA  171 non-null    object
 13  TIPO_DOC_
SOLICITANTE     151 non-null    object
 14  NRO_DOC_
SOLICITANTE      

Let's keep the first one:

## 1. Clean headers

In [8]:
# check headers
df_ordenado.columns

Index(['CODIGO_\nENTIDAD', 'CODIGO_\nUBIGEO', 'CODIGO_\nPAIS', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'NOMBRE_UO', 'GOBIERNO_LOCAL',
       'RUC_GOBIERNO\n_LOCAL', 'NUMERO_\nEXPEDIENTE',
       'FECHA_\nINICIO_\nTRAMITE', 'FECHA_\nEMISION_TRAMITE',
       'NRO_RESOLUCION_\nLICENCIA', 'TIPO_DOC_\nSOLICITANTE',
       'NRO_DOC_\nSOLICITANTE', 'UBICACION_PREDIO', 'ZONIFICACION_\nPREDIO',
       'TIPO_DE_USO', 'TIPO_ALTURA', 'DETALLE_ALTURA',
       'MODALIDAD_\nAPROBACION', '\nTIPO_OBRA', 'OBRA_\nNUEVA',
       'OBRA_\nDEMOLICIÓN ', 'OBRA_\nREMODELACIÓN ', 'OBRA_\nAMPLIACIÓN ',
       'OBRA_\nREFACCIÓN ', 'OBRA_\nCERCO ', 'AREA_TOTAL_\nCONSTRUIDA ',
       'VALOR_\nOBRA', 'FECHA_\nCORTE'],
      dtype='object')

Cleaning requires a strategy. In the strings above your main problem is the footnotes and the quasi-duplicates.

In [9]:
# the quasi duplicates
nColumns=df_ordenado.columns[df_ordenado.columns.str.contains('_\n')]
nColumns

Index(['CODIGO_\nENTIDAD', 'CODIGO_\nUBIGEO', 'CODIGO_\nPAIS',
       'NUMERO_\nEXPEDIENTE', 'FECHA_\nINICIO_\nTRAMITE',
       'FECHA_\nEMISION_TRAMITE', 'NRO_RESOLUCION_\nLICENCIA',
       'TIPO_DOC_\nSOLICITANTE', 'NRO_DOC_\nSOLICITANTE',
       'ZONIFICACION_\nPREDIO', 'MODALIDAD_\nAPROBACION', 'OBRA_\nNUEVA',
       'OBRA_\nDEMOLICIÓN ', 'OBRA_\nREMODELACIÓN ', 'OBRA_\nAMPLIACIÓN ',
       'OBRA_\nREFACCIÓN ', 'OBRA_\nCERCO ', 'AREA_TOTAL_\nCONSTRUIDA ',
       'VALOR_\nOBRA', 'FECHA_\nCORTE'],
      dtype='object')

In [10]:
# the not quasi duplicates
df_ordenado.columns[~df_ordenado.columns.str.contains('_\n')]

Index(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'NOMBRE_UO', 'GOBIERNO_LOCAL',
       'RUC_GOBIERNO\n_LOCAL', 'UBICACION_PREDIO', 'TIPO_DE_USO',
       'TIPO_ALTURA', 'DETALLE_ALTURA', '\nTIPO_OBRA'],
      dtype='object')

Let's keep the last ones without the footnotes, let's _divide and conquer_ using **split()**:

In [11]:
# save the last one but the first ones:
notnColumns=df_ordenado.columns[~df_ordenado.columns.str.contains('\n')][3:]
notnColumns

Index(['NOMBRE_UO', 'GOBIERNO_LOCAL', 'UBICACION_PREDIO', 'TIPO_DE_USO',
       'TIPO_ALTURA', 'DETALLE_ALTURA'],
      dtype='object')

NOT N COLUMNS

In [12]:
# using list comprehension
notnColumns1=[element.split('_') for element in notnColumns]
notnColumns1

[['NOMBRE', 'UO'],
 ['GOBIERNO', 'LOCAL'],
 ['UBICACION', 'PREDIO'],
 ['TIPO', 'DE', 'USO'],
 ['TIPO', 'ALTURA'],
 ['DETALLE', 'ALTURA']]

In [13]:
notnColumns2 = []

In [14]:
for element in notnColumns1:
    nElement = ' '.join(element)
    notnColumns2.append(nElement)
notnColumns2

['NOMBRE UO',
 'GOBIERNO LOCAL',
 'UBICACION PREDIO',
 'TIPO DE USO',
 'TIPO ALTURA',
 'DETALLE ALTURA']

RUC GOBIERNO LOCAL

In [15]:
RUCnColumns=df_ordenado.columns[df_ordenado.columns.str.contains('RUC')]
RUCnColumns

Index(['RUC_GOBIERNO\n_LOCAL'], dtype='object')

In [16]:
RUCnColumns1=[element.split('\n_') for element in RUCnColumns]
RUCnColumns1

[['RUC_GOBIERNO', 'LOCAL']]

In [17]:
RUCnColumns2 = []

In [18]:
for element in RUCnColumns1:
    RUCElement = ' '.join(element)
    RUCnColumns2.append(RUCElement)
RUCnColumns2

['RUC_GOBIERNO LOCAL']

In [19]:
RUCnColumns3=[element.split('_') for element in RUCnColumns2]
RUCnColumns3

[['RUC', 'GOBIERNO LOCAL']]

In [20]:
RUCnColumns4 = []

In [21]:
for element in RUCnColumns3:
    RUCElement1 = ' '.join(element)
    RUCnColumns4.append(RUCElement1)
RUCnColumns4

['RUC GOBIERNO LOCAL']

TIPO DE OBRA

In [22]:
OBRAnColumns=df_ordenado.columns[df_ordenado.columns.str.contains('TIPO_OBRA')]
OBRAnColumns

Index(['\nTIPO_OBRA'], dtype='object')

In [23]:
OBRAnColumns1=[element.strip('\n') for element in OBRAnColumns]
OBRAnColumns1

['TIPO_OBRA']

In [24]:
OBRAnColumns2=[element.split('_') for element in OBRAnColumns1]
OBRAnColumns2

[['TIPO', 'OBRA']]

In [25]:
OBRAnColumns3 = []

In [26]:
for element in OBRAnColumns2:
    OBRAElement1 = ' '.join(element)
    OBRAnColumns3.append(OBRAElement1)
OBRAnColumns3

['TIPO OBRA']

N COLUMNS

In [27]:
nColumnsa=[element.strip(' ') for element in nColumns]
nColumnsa

['CODIGO_\nENTIDAD',
 'CODIGO_\nUBIGEO',
 'CODIGO_\nPAIS',
 'NUMERO_\nEXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE',
 'FECHA_\nEMISION_TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA',
 'TIPO_DOC_\nSOLICITANTE',
 'NRO_DOC_\nSOLICITANTE',
 'ZONIFICACION_\nPREDIO',
 'MODALIDAD_\nAPROBACION',
 'OBRA_\nNUEVA',
 'OBRA_\nDEMOLICIÓN',
 'OBRA_\nREMODELACIÓN',
 'OBRA_\nAMPLIACIÓN',
 'OBRA_\nREFACCIÓN',
 'OBRA_\nCERCO',
 'AREA_TOTAL_\nCONSTRUIDA',
 'VALOR_\nOBRA',
 'FECHA_\nCORTE']

In [28]:
nColumns1=[element.split('_\n') for element in nColumnsa]
nColumns1

[['CODIGO', 'ENTIDAD'],
 ['CODIGO', 'UBIGEO'],
 ['CODIGO', 'PAIS'],
 ['NUMERO', 'EXPEDIENTE'],
 ['FECHA', 'INICIO', 'TRAMITE'],
 ['FECHA', 'EMISION_TRAMITE'],
 ['NRO_RESOLUCION', 'LICENCIA'],
 ['TIPO_DOC', 'SOLICITANTE'],
 ['NRO_DOC', 'SOLICITANTE'],
 ['ZONIFICACION', 'PREDIO'],
 ['MODALIDAD', 'APROBACION'],
 ['OBRA', 'NUEVA'],
 ['OBRA', 'DEMOLICIÓN'],
 ['OBRA', 'REMODELACIÓN'],
 ['OBRA', 'AMPLIACIÓN'],
 ['OBRA', 'REFACCIÓN'],
 ['OBRA', 'CERCO'],
 ['AREA_TOTAL', 'CONSTRUIDA'],
 ['VALOR', 'OBRA'],
 ['FECHA', 'CORTE']]

In [29]:
nColumns2 = []

In [30]:
for element in nColumns1:
    nElement2 = ' '.join(element)
    nColumns2.append(nElement2)
nColumns2

['CODIGO ENTIDAD',
 'CODIGO UBIGEO',
 'CODIGO PAIS',
 'NUMERO EXPEDIENTE',
 'FECHA INICIO TRAMITE',
 'FECHA EMISION_TRAMITE',
 'NRO_RESOLUCION LICENCIA',
 'TIPO_DOC SOLICITANTE',
 'NRO_DOC SOLICITANTE',
 'ZONIFICACION PREDIO',
 'MODALIDAD APROBACION',
 'OBRA NUEVA',
 'OBRA DEMOLICIÓN',
 'OBRA REMODELACIÓN',
 'OBRA AMPLIACIÓN',
 'OBRA REFACCIÓN',
 'OBRA CERCO',
 'AREA_TOTAL CONSTRUIDA',
 'VALOR OBRA',
 'FECHA CORTE']

In [31]:
nColumns3=[element.split('_') for element in nColumns2]
nColumns3

[['CODIGO ENTIDAD'],
 ['CODIGO UBIGEO'],
 ['CODIGO PAIS'],
 ['NUMERO EXPEDIENTE'],
 ['FECHA INICIO TRAMITE'],
 ['FECHA EMISION', 'TRAMITE'],
 ['NRO', 'RESOLUCION LICENCIA'],
 ['TIPO', 'DOC SOLICITANTE'],
 ['NRO', 'DOC SOLICITANTE'],
 ['ZONIFICACION PREDIO'],
 ['MODALIDAD APROBACION'],
 ['OBRA NUEVA'],
 ['OBRA DEMOLICIÓN'],
 ['OBRA REMODELACIÓN'],
 ['OBRA AMPLIACIÓN'],
 ['OBRA REFACCIÓN'],
 ['OBRA CERCO'],
 ['AREA', 'TOTAL CONSTRUIDA'],
 ['VALOR OBRA'],
 ['FECHA CORTE']]

In [32]:
nColumns4 = []

In [33]:
for element in nColumns3:
    elementn = ' '.join(element)
    nColumns4.append(elementn)
nColumns4

['CODIGO ENTIDAD',
 'CODIGO UBIGEO',
 'CODIGO PAIS',
 'NUMERO EXPEDIENTE',
 'FECHA INICIO TRAMITE',
 'FECHA EMISION TRAMITE',
 'NRO RESOLUCION LICENCIA',
 'TIPO DOC SOLICITANTE',
 'NRO DOC SOLICITANTE',
 'ZONIFICACION PREDIO',
 'MODALIDAD APROBACION',
 'OBRA NUEVA',
 'OBRA DEMOLICIÓN',
 'OBRA REMODELACIÓN',
 'OBRA AMPLIACIÓN',
 'OBRA REFACCIÓN',
 'OBRA CERCO',
 'AREA TOTAL CONSTRUIDA',
 'VALOR OBRA',
 'FECHA CORTE']

In [34]:
change={old:new for old,new in zip(notnColumns,notnColumns2)}
change

{'NOMBRE_UO': 'NOMBRE UO',
 'GOBIERNO_LOCAL': 'GOBIERNO LOCAL',
 'UBICACION_PREDIO': 'UBICACION PREDIO',
 'TIPO_DE_USO': 'TIPO DE USO',
 'TIPO_ALTURA': 'TIPO ALTURA',
 'DETALLE_ALTURA': 'DETALLE ALTURA'}

In [35]:
change2={old:new for old,new in zip(nColumns,nColumns4)}
change2

{'CODIGO_\nENTIDAD': 'CODIGO ENTIDAD',
 'CODIGO_\nUBIGEO': 'CODIGO UBIGEO',
 'CODIGO_\nPAIS': 'CODIGO PAIS',
 'NUMERO_\nEXPEDIENTE': 'NUMERO EXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE': 'FECHA INICIO TRAMITE',
 'FECHA_\nEMISION_TRAMITE': 'FECHA EMISION TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA': 'NRO RESOLUCION LICENCIA',
 'TIPO_DOC_\nSOLICITANTE': 'TIPO DOC SOLICITANTE',
 'NRO_DOC_\nSOLICITANTE': 'NRO DOC SOLICITANTE',
 'ZONIFICACION_\nPREDIO': 'ZONIFICACION PREDIO',
 'MODALIDAD_\nAPROBACION': 'MODALIDAD APROBACION',
 'OBRA_\nNUEVA': 'OBRA NUEVA',
 'OBRA_\nDEMOLICIÓN ': 'OBRA DEMOLICIÓN',
 'OBRA_\nREMODELACIÓN ': 'OBRA REMODELACIÓN',
 'OBRA_\nAMPLIACIÓN ': 'OBRA AMPLIACIÓN',
 'OBRA_\nREFACCIÓN ': 'OBRA REFACCIÓN',
 'OBRA_\nCERCO ': 'OBRA CERCO',
 'AREA_TOTAL_\nCONSTRUIDA ': 'AREA TOTAL CONSTRUIDA',
 'VALOR_\nOBRA': 'VALOR OBRA',
 'FECHA_\nCORTE': 'FECHA CORTE'}

In [36]:
change3={old:new for old,new in zip(OBRAnColumns,OBRAnColumns3)}
change3

{'\nTIPO_OBRA': 'TIPO OBRA'}

In [37]:
change4={old:new for old,new in zip(RUCnColumns,RUCnColumns4)}
change4

{'RUC_GOBIERNO\n_LOCAL': 'RUC GOBIERNO LOCAL'}

In [38]:
change.update(change2)
change

{'NOMBRE_UO': 'NOMBRE UO',
 'GOBIERNO_LOCAL': 'GOBIERNO LOCAL',
 'UBICACION_PREDIO': 'UBICACION PREDIO',
 'TIPO_DE_USO': 'TIPO DE USO',
 'TIPO_ALTURA': 'TIPO ALTURA',
 'DETALLE_ALTURA': 'DETALLE ALTURA',
 'CODIGO_\nENTIDAD': 'CODIGO ENTIDAD',
 'CODIGO_\nUBIGEO': 'CODIGO UBIGEO',
 'CODIGO_\nPAIS': 'CODIGO PAIS',
 'NUMERO_\nEXPEDIENTE': 'NUMERO EXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE': 'FECHA INICIO TRAMITE',
 'FECHA_\nEMISION_TRAMITE': 'FECHA EMISION TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA': 'NRO RESOLUCION LICENCIA',
 'TIPO_DOC_\nSOLICITANTE': 'TIPO DOC SOLICITANTE',
 'NRO_DOC_\nSOLICITANTE': 'NRO DOC SOLICITANTE',
 'ZONIFICACION_\nPREDIO': 'ZONIFICACION PREDIO',
 'MODALIDAD_\nAPROBACION': 'MODALIDAD APROBACION',
 'OBRA_\nNUEVA': 'OBRA NUEVA',
 'OBRA_\nDEMOLICIÓN ': 'OBRA DEMOLICIÓN',
 'OBRA_\nREMODELACIÓN ': 'OBRA REMODELACIÓN',
 'OBRA_\nAMPLIACIÓN ': 'OBRA AMPLIACIÓN',
 'OBRA_\nREFACCIÓN ': 'OBRA REFACCIÓN',
 'OBRA_\nCERCO ': 'OBRA CERCO',
 'AREA_TOTAL_\nCONSTRUIDA ': 'AREA TOTAL CONST

In [39]:
change.update(change3)
change

{'NOMBRE_UO': 'NOMBRE UO',
 'GOBIERNO_LOCAL': 'GOBIERNO LOCAL',
 'UBICACION_PREDIO': 'UBICACION PREDIO',
 'TIPO_DE_USO': 'TIPO DE USO',
 'TIPO_ALTURA': 'TIPO ALTURA',
 'DETALLE_ALTURA': 'DETALLE ALTURA',
 'CODIGO_\nENTIDAD': 'CODIGO ENTIDAD',
 'CODIGO_\nUBIGEO': 'CODIGO UBIGEO',
 'CODIGO_\nPAIS': 'CODIGO PAIS',
 'NUMERO_\nEXPEDIENTE': 'NUMERO EXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE': 'FECHA INICIO TRAMITE',
 'FECHA_\nEMISION_TRAMITE': 'FECHA EMISION TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA': 'NRO RESOLUCION LICENCIA',
 'TIPO_DOC_\nSOLICITANTE': 'TIPO DOC SOLICITANTE',
 'NRO_DOC_\nSOLICITANTE': 'NRO DOC SOLICITANTE',
 'ZONIFICACION_\nPREDIO': 'ZONIFICACION PREDIO',
 'MODALIDAD_\nAPROBACION': 'MODALIDAD APROBACION',
 'OBRA_\nNUEVA': 'OBRA NUEVA',
 'OBRA_\nDEMOLICIÓN ': 'OBRA DEMOLICIÓN',
 'OBRA_\nREMODELACIÓN ': 'OBRA REMODELACIÓN',
 'OBRA_\nAMPLIACIÓN ': 'OBRA AMPLIACIÓN',
 'OBRA_\nREFACCIÓN ': 'OBRA REFACCIÓN',
 'OBRA_\nCERCO ': 'OBRA CERCO',
 'AREA_TOTAL_\nCONSTRUIDA ': 'AREA TOTAL CONST

In [40]:
change.update(change4)
change

{'NOMBRE_UO': 'NOMBRE UO',
 'GOBIERNO_LOCAL': 'GOBIERNO LOCAL',
 'UBICACION_PREDIO': 'UBICACION PREDIO',
 'TIPO_DE_USO': 'TIPO DE USO',
 'TIPO_ALTURA': 'TIPO ALTURA',
 'DETALLE_ALTURA': 'DETALLE ALTURA',
 'CODIGO_\nENTIDAD': 'CODIGO ENTIDAD',
 'CODIGO_\nUBIGEO': 'CODIGO UBIGEO',
 'CODIGO_\nPAIS': 'CODIGO PAIS',
 'NUMERO_\nEXPEDIENTE': 'NUMERO EXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE': 'FECHA INICIO TRAMITE',
 'FECHA_\nEMISION_TRAMITE': 'FECHA EMISION TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA': 'NRO RESOLUCION LICENCIA',
 'TIPO_DOC_\nSOLICITANTE': 'TIPO DOC SOLICITANTE',
 'NRO_DOC_\nSOLICITANTE': 'NRO DOC SOLICITANTE',
 'ZONIFICACION_\nPREDIO': 'ZONIFICACION PREDIO',
 'MODALIDAD_\nAPROBACION': 'MODALIDAD APROBACION',
 'OBRA_\nNUEVA': 'OBRA NUEVA',
 'OBRA_\nDEMOLICIÓN ': 'OBRA DEMOLICIÓN',
 'OBRA_\nREMODELACIÓN ': 'OBRA REMODELACIÓN',
 'OBRA_\nAMPLIACIÓN ': 'OBRA AMPLIACIÓN',
 'OBRA_\nREFACCIÓN ': 'OBRA REFACCIÓN',
 'OBRA_\nCERCO ': 'OBRA CERCO',
 'AREA_TOTAL_\nCONSTRUIDA ': 'AREA TOTAL CONST

In [41]:
change

{'NOMBRE_UO': 'NOMBRE UO',
 'GOBIERNO_LOCAL': 'GOBIERNO LOCAL',
 'UBICACION_PREDIO': 'UBICACION PREDIO',
 'TIPO_DE_USO': 'TIPO DE USO',
 'TIPO_ALTURA': 'TIPO ALTURA',
 'DETALLE_ALTURA': 'DETALLE ALTURA',
 'CODIGO_\nENTIDAD': 'CODIGO ENTIDAD',
 'CODIGO_\nUBIGEO': 'CODIGO UBIGEO',
 'CODIGO_\nPAIS': 'CODIGO PAIS',
 'NUMERO_\nEXPEDIENTE': 'NUMERO EXPEDIENTE',
 'FECHA_\nINICIO_\nTRAMITE': 'FECHA INICIO TRAMITE',
 'FECHA_\nEMISION_TRAMITE': 'FECHA EMISION TRAMITE',
 'NRO_RESOLUCION_\nLICENCIA': 'NRO RESOLUCION LICENCIA',
 'TIPO_DOC_\nSOLICITANTE': 'TIPO DOC SOLICITANTE',
 'NRO_DOC_\nSOLICITANTE': 'NRO DOC SOLICITANTE',
 'ZONIFICACION_\nPREDIO': 'ZONIFICACION PREDIO',
 'MODALIDAD_\nAPROBACION': 'MODALIDAD APROBACION',
 'OBRA_\nNUEVA': 'OBRA NUEVA',
 'OBRA_\nDEMOLICIÓN ': 'OBRA DEMOLICIÓN',
 'OBRA_\nREMODELACIÓN ': 'OBRA REMODELACIÓN',
 'OBRA_\nAMPLIACIÓN ': 'OBRA AMPLIACIÓN',
 'OBRA_\nREFACCIÓN ': 'OBRA REFACCIÓN',
 'OBRA_\nCERCO ': 'OBRA CERCO',
 'AREA_TOTAL_\nCONSTRUIDA ': 'AREA TOTAL CONST

In [42]:
df_ordenado.rename(columns=change,inplace=True)
df_ordenado.head()

,CODIGO ENTIDAD,CODIGO UBIGEO,CODIGO PAIS,DEPARTAMENTO,PROVINCIA,DISTRITO,NOMBRE UO,GOBIERNO LOCAL,RUC GOBIERNO LOCAL,NUMERO EXPEDIENTE,...,TIPO OBRA,OBRA NUEVA,OBRA DEMOLICIÓN,OBRA REMODELACIÓN,OBRA AMPLIACIÓN,OBRA REFACCIÓN,OBRA CERCO,AREA TOTAL CONSTRUIDA,VALOR OBRA,FECHA CORTE
0,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,15636-2022,...,AMPLIACIÓN/REMODELACIÓN,-,-,"97,83","208,46",-,-,"208,46","225,652,12",20221231
115,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,7494-2022,...,LICENCIA DE DEMOLICION TOTAL,-,200,-,-,-,-,-,45455.7,20221231
116,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,3732-2022,...,LICENCIA DE EDIFICACION NUEVA,1688.79,-,-,-,-,-,1688.79,1561032.09,20221231
117,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,3947-2022,...,LICENCIA DE EDIFICACION NUEVA,1950.08,-,-,-,-,-,1950.08,1885715.08,20221231
118,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,3372-2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20221231


## 2. Clean the data values.

Since there are categories, we could try some frequency tables:

In [43]:
df_ordenado.columns

Index(['CODIGO ENTIDAD', 'CODIGO UBIGEO', 'CODIGO PAIS', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'NOMBRE UO', 'GOBIERNO LOCAL',
       'RUC GOBIERNO LOCAL', 'NUMERO EXPEDIENTE', 'FECHA INICIO TRAMITE',
       'FECHA EMISION TRAMITE', 'NRO RESOLUCION LICENCIA',
       'TIPO DOC SOLICITANTE', 'NRO DOC SOLICITANTE', 'UBICACION PREDIO',
       'ZONIFICACION PREDIO', 'TIPO DE USO', 'TIPO ALTURA', 'DETALLE ALTURA',
       'MODALIDAD APROBACION', 'TIPO OBRA', 'OBRA NUEVA', 'OBRA DEMOLICIÓN',
       'OBRA REMODELACIÓN', 'OBRA AMPLIACIÓN', 'OBRA REFACCIÓN', 'OBRA CERCO',
       'AREA TOTAL CONSTRUIDA', 'VALOR OBRA', 'FECHA CORTE'],
      dtype='object')

Código Entidad

In [44]:
df_ordenado['CODIGO ENTIDAD'].value_counts(dropna=False).sort_index()

301285    180
Name: CODIGO ENTIDAD, dtype: int64

Código Ubigeo

In [45]:
df_ordenado['CODIGO UBIGEO'].value_counts(dropna=False).sort_index()

150136    180
Name: CODIGO UBIGEO, dtype: int64

Código País

In [46]:
df_ordenado['CODIGO PAIS'].value_counts(dropna=False).sort_index()

PE    180
Name: CODIGO PAIS, dtype: int64

Departamento

In [47]:
df_ordenado['DEPARTAMENTO'].value_counts(dropna=False).sort_index()

Lima    180
Name: DEPARTAMENTO, dtype: int64

Provincia

In [48]:
df_ordenado['PROVINCIA'].value_counts(dropna=False).sort_index()

Lima    180
Name: PROVINCIA, dtype: int64

Distrito

In [49]:
df_ordenado['DISTRITO'].value_counts(dropna=False).sort_index()

San Miguel    180
Name: DISTRITO, dtype: int64

Nombre Unidad Orgánica

In [50]:
df_ordenado['NOMBRE UO'].value_counts(dropna=False).sort_index()

Subgerencia de Obras Privadas    180
Name: NOMBRE UO, dtype: int64

Gobierno Local

In [51]:
df_ordenado['GOBIERNO LOCAL'].value_counts(dropna=False).sort_index()

San Miguel    180
Name: GOBIERNO LOCAL, dtype: int64

RUC Gobierno Local

In [52]:
df_ordenado['RUC GOBIERNO LOCAL'].value_counts(dropna=False).sort_index()

20131372184    180
Name: RUC GOBIERNO LOCAL, dtype: int64

Número Expediente

In [53]:
df_ordenado['NUMERO EXPEDIENTE'].value_counts(dropna=False).sort_index()

00279-2022    1
059-2021      1
0637-2022     1
10019-2022    1
10101-2022    1
             ..
9419-2022     1
9457-2022     1
9485-2022     1
9767-2021     1
9935-2022     1
Name: NUMERO EXPEDIENTE, Length: 179, dtype: int64

Número de la resolución de licencia

In [54]:
df_ordenado['NRO RESOLUCION LICENCIA'] = df_ordenado['NRO RESOLUCION LICENCIA'].astype(str)

In [55]:
df_ordenado['NRO RESOLUCION LICENCIA'].value_counts(dropna=False).sort_index()

0053-2022    1
0069-2022    1
0072-2022    1
009-2022     1
040-2022     1
            ..
97-2022      1
991-2022     1
994-2022     1
995-2022     1
nan          9
Name: NRO RESOLUCION LICENCIA, Length: 171, dtype: int64

Tipo Documento Solicitante

In [56]:
df_ordenado['TIPO DOC SOLICITANTE'].value_counts(dropna=False).sort_index()

DNI     59
DNI      4
RDA      1
RUC     71
RUC     16
NaN     29
Name: TIPO DOC SOLICITANTE, dtype: int64

In [57]:
df_ordenado['TIPO DOC SOLICITANTE'] = df_ordenado['TIPO DOC SOLICITANTE'].str.strip()

In [58]:
df_ordenado['TIPO DOC SOLICITANTE'].value_counts(dropna=False).sort_index()

DNI    63
RDA     1
RUC    87
NaN    29
Name: TIPO DOC SOLICITANTE, dtype: int64

Número Documento Solicitante

In [59]:
df_ordenado['NRO DOC SOLICITANTE'] = pd.to_numeric(df_ordenado['NRO DOC SOLICITANTE'], errors='coerce')
df_ordenado= df_ordenado.dropna(subset=['NRO DOC SOLICITANTE'])
df_ordenado['NRO DOC SOLICITANTE'].astype(int)


0      20514554014
115    20608254421
116     2054867913
117    20555184086
119    20605597204
          ...     
61         6862888
63     20604661880
64     20604661880
66     20600986164
179    20605359737
Name: NRO DOC SOLICITANTE, Length: 117, dtype: int64

In [60]:
df_ordenado['NRO DOC SOLICITANTE'].value_counts(dropna=False).sort_index()

8.777210e+05    1
6.055174e+06    1
6.134161e+06    1
6.720981e+06    1
6.862888e+06    1
               ..
2.060887e+10    1
2.060889e+10    2
2.060893e+10    1
2.060906e+10    1
2.050000e+11    1
Name: NRO DOC SOLICITANTE, Length: 89, dtype: int64

Ubicación Predio

In [61]:
df_ordenado['UBICACION PREDIO'].value_counts(dropna=False).sort_index()

AV COSTANERA  2982 - -                                                      1
AV COSTANERA 2560 -- -                                                      1
AV LA MARINA  2115 -- -                                                     1
AV LA MARINA  2201-2235 -P-1-2 40429                                        1
AV LA MARINA / MONSEÑOR J. DINTILHAC 2201-2235/585 -P-1-2 8 - 9 - 10 -11    1
                                                                           ..
JR. MANCO II - -- 5                                                         1
JR. TALAMBO 161 -- -                                                        1
PASAJE  VIRGEN DE FATIMA  233 -- -                                          1
PASAJE LAS ROSAS 179 -- -                                                   1
PASAJE UNO - -A 14                                                          1
Name: UBICACION PREDIO, Length: 113, dtype: int64

In [62]:
df_ordenado['UBICACION PREDIO'] = df_ordenado['UBICACION PREDIO'].str.strip()

<ipython-input-62-0add9f28a840>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ordenado['UBICACION PREDIO'] = df_ordenado['UBICACION PREDIO'].str.strip()


In [63]:
df_ordenado['UBICACION PREDIO'].value_counts(dropna=False).sort_index()

AV COSTANERA  2982 - -                                                      1
AV COSTANERA 2560 -- -                                                      1
AV LA MARINA  2115 -- -                                                     1
AV LA MARINA  2201-2235 -P-1-2 40429                                        1
AV LA MARINA / MONSEÑOR J. DINTILHAC 2201-2235/585 -P-1-2 8 - 9 - 10 -11    1
                                                                           ..
JR. MANCO II - -- 5                                                         1
JR. TALAMBO 161 -- -                                                        1
PASAJE  VIRGEN DE FATIMA  233 -- -                                          1
PASAJE LAS ROSAS 179 -- -                                                   1
PASAJE UNO - -A 14                                                          1
Name: UBICACION PREDIO, Length: 113, dtype: int64

Zonificación Predio

In [64]:
df_ordenado['ZONIFICACION PREDIO'].value_counts(dropna=False).sort_index()

CM            4
CZ            6
CZ            1
E1            3
RDA          20
RDA Y RDM     1
RDM          69
RDM          11
RDM-CV        1
NaN           1
Name: ZONIFICACION PREDIO, dtype: int64

In [65]:
df_ordenado['ZONIFICACION PREDIO'] = df_ordenado['ZONIFICACION PREDIO'].str.strip()

In [66]:
df_ordenado['ZONIFICACION PREDIO'].value_counts(dropna=False).sort_index()

CM            4
CZ            7
E1            3
RDA          20
RDA Y RDM     1
RDM          80
RDM-CV        1
NaN           1
Name: ZONIFICACION PREDIO, dtype: int64

Tipo de Uso

In [67]:
USO= df_ordenado['TIPO DE USO'].value_counts(dropna=False).sort_index()

In [68]:
df_ordenado['TIPO DE USO'] = df_ordenado['TIPO DE USO'].str.strip()

In [69]:
df_ordenado['TIPO DE USO'].value_counts(dropna=False).sort_index()

CERCO                                                 1
COMERCIO                                             11
CONJUNTO RESIDENCIAL                                  4
E1                                                    1
EDIFICIO MULTIFAMILIAR - MODIFICACION DE PROYECTO     1
EDUCACION                                             1
MULTIFAMILIAR                                        45
MULTIFAMILIAR - COMERCIO                              2
TERRENO                                               1
UNIFAMILIAR                                           6
VIIENDA UNIFAMILIAR                                   1
VIV.MULTIFAMILIAR                                     1
VIVIENA UNIFAMILIAR                                   1
VIVIENDA - COMERCIO                                   2
VIVIENDA MULTIFAMILIAR                               14
VIVIENDA UNIFAMILIAR                                 23
VIVIUENDA UNIFAMILIAR                                 1
NaN                                             

In [70]:
def UNIFORMIDAD(FAMILIA):
    if pd.isnull(FAMILIA):
        return FAMILIA
    FAMILIA = FAMILIA.upper()
    if 'UNIFAMILIAR' in FAMILIA:
        return 'UNIFAMILIAR'
    elif 'MULTIFAMILIAR' in FAMILIA:
        return 'MULTIFAMILIAR'
    elif "COMERCIO" in FAMILIA:
        return "COMERCIO"
    else:
        return FAMILIA

df_ordenado['TIPO DE USO NORMALIZADO'] = df_ordenado['TIPO DE USO'].apply(UNIFORMIDAD)

USO1= df_ordenado['TIPO DE USO NORMALIZADO'].value_counts(dropna=False).sort_index()
USO1

CERCO                    1
COMERCIO                13
CONJUNTO RESIDENCIAL     4
E1                       1
EDUCACION                1
MULTIFAMILIAR           63
TERRENO                  1
UNIFAMILIAR             32
NaN                      1
Name: TIPO DE USO NORMALIZADO, dtype: int64

Tipo de Altura

In [71]:
df_ordenado['TIPO ALTURA'].value_counts(dropna=False).sort_index()

PISOS     26
PISOS     91
Name: TIPO ALTURA, dtype: int64

In [72]:
df_ordenado['TIPO ALTURA'] = df_ordenado['TIPO ALTURA'].str.strip()

In [73]:
df_ordenado['TIPO ALTURA'].value_counts(dropna=False).sort_index()

PISOS    117
Name: TIPO ALTURA, dtype: int64

Detalle altura

In [74]:
df_ordenado['DETALLE ALTURA'] = df_ordenado['DETALLE ALTURA'].astype(str)

In [75]:
df_ordenado['DETALLE ALTURA'].value_counts(dropna=False).sort_index()

 3 PISOS + AZOTEA                  1
0                                  1
1 PISO                             6
1 PISO                             3
1 SOTANO+7 PISOS+AZOTEA            1
                                  ..
SOTANO+SEMISOTANO+6P+AZ            1
SOTANO+SEMISOTANO+6PISOS+AZOTEA    1
SOTANO+SEMISOTANO+8P+AZ            1
SOTANO+SEMISOTANO+8PISOS           1
nan                                1
Name: DETALLE ALTURA, Length: 67, dtype: int64

In [76]:
df_ordenado['DETALLE ALTURA'] = df_ordenado['DETALLE ALTURA'].str.strip()

In [77]:
df_ordenado['DETALLE ALTURA'].value_counts(dropna=False).sort_index()

0                                  1
1 PISO                             9
1 SOTANO+7 PISOS+AZOTEA            1
11P+AZ                             1
13:00:00                           1
                                  ..
SOTANO+SEMISOTANO+6P+AZ            1
SOTANO+SEMISOTANO+6PISOS+AZOTEA    1
SOTANO+SEMISOTANO+8P+AZ            1
SOTANO+SEMISOTANO+8PISOS           1
nan                                1
Name: DETALLE ALTURA, Length: 64, dtype: int64

In [78]:
def ALTURA(DETALLE):
    if '1P' in DETALLE:
        return '1 PISO'
    elif '2P' in DETALLE:
        return '2 PISOS'
    elif "3P" in DETALLE:
        return "3 PISOS"
    elif "4P" in DETALLE:
        return "4 PISOS"
    elif "5P" in DETALLE:
        return "5 PISOS"
    else:
        return DETALLE

df_ordenado['DETALLE ALTURA'] = df_ordenado['DETALLE ALTURA'].apply(ALTURA)

df_ordenado['DETALLE ALTURA'].value_counts(dropna=False).sort_index()

0                                              1
1 PISO                                        14
1 SOTANO+7 PISOS+AZOTEA                        1
13:00:00                                       1
1SOTANO+17PISOS+AZOTEA                         1
2 PISO                                         1
2 PISOS                                       18
2 PISOS + AZ                                   1
2 PISOS + AZOTEA                               4
2 SOTANOS + 7 PISOS + AZOTEA                   1
2 SOTANOS + SEMISOTANO + 10 PISOS + AZOTEA     1
2 SOTANOS+17P+AZ                               1
2SOT.+16PISOS+AZO.                             1
2SOTANOS+7PISOS+AZOTEA                         1
3 PISO                                         1
3 PISOS                                       14
3 PISOS + AZOTEA                               4
3 SOT,+20 PISOS+AZOTEA                         1
3 SOT.+18 PISOS+AZO.                           1
3 SÓTANOS + 9 PISOS + AZOTEA                   1
4 PISOS             

In [79]:
def ALTURA2(DETALLE2):
    if '1 PISO' in DETALLE2:
        return '1 PISO'
    elif '2 PISO' in DETALLE2:
        return '2 PISOS'
    elif "3 PISO" in DETALLE2:
        return "3 PISOS"
    elif "4 PISO" in DETALLE2:
        return "4 PISOS"
    elif "5 PISO" in DETALLE2:
        return "5 PISOS"
    elif "6P" in DETALLE2:
        return "6 PISOS"
    elif "6 PISO" in DETALLE2:
        return "6 PISOS"
    elif "7 PISO" in DETALLE2:
        return "7 PISOS"
    elif "7P" in DETALLE2:
        return "7 PISOS"
    elif "8 PISO" in DETALLE2:
        return "8 PISOS"
    elif "8P" in DETALLE2:
        return "8 PISOS"
    elif "9 PISO" in DETALLE2:
        return "9 PISOS"
    elif "9P" in DETALLE2:
        return "9 PISOS"
    elif "10 PISO" in DETALLE2:
        return "10 PISOS"
    elif "10P" in DETALLE2:
        return "10 PISOS"
    elif "11 PISO" in DETALLE2:
        return "11 PISOS"
    elif "12 PISO" in DETALLE2:
        return "12 PISOS"
    elif "13 PISO" in DETALLE2:
        return "13 PISOS"
    elif "14 PISO" in DETALLE2:
        return "14 PISOS"
    elif "15 PISO" in DETALLE2:
        return "15 PISOS"
    elif "16 PISO" in DETALLE2:
        return "16 PISOS"
    elif "17 PISO" in DETALLE2:
        return "17 PISOS"
    elif "17P" in DETALLE2:
        return "17 PISOS"
    elif "20 PISOS" in DETALLE2:
        return "20 PISOS"
    elif "20P" in DETALLE2:
        return "20 PISOS"
    elif "13:0" in DETALLE2:
        return ""
    else:
        return DETALLE2

df_ordenado['DETALLE ALTURA'] = df_ordenado['DETALLE ALTURA'].apply(ALTURA2)

df_ordenado['DETALLE ALTURA'].value_counts(dropna=False).sort_index()

             1
0            1
1 PISO      14
10 PISOS     2
2 PISOS     24
20 PISOS     5
3 PISOS     19
4 PISOS      6
5 PISOS     13
6 PISOS      9
7 PISOS     12
8 PISOS      6
9 PISOS      4
nan          1
Name: DETALLE ALTURA, dtype: int64

Modalidad Aprobación

In [80]:
df_ordenado['MODALIDAD APROBACION'] = df_ordenado['MODALIDAD APROBACION'].astype(str)

In [81]:
df_ordenado['MODALIDAD APROBACION'].value_counts(dropna=False).sort_index()

" A"                       1
A                         31
B                         31
C                         32
C                          6
C  REVISORES URBANOS       6
C - REVISORES URBANOS      1
C - REVISORES URBANOS      3
C REVISORES URBANOS        1
D                          2
E1                         2
nan                        1
Name: MODALIDAD APROBACION, dtype: int64

In [82]:
def APROBACION(MODALIDAD):
    if 'C' in MODALIDAD:
        return 'C'
    if "A" in MODALIDAD:
        return "A"
    else:
        return MODALIDAD

df_ordenado['MODALIDAD APROBACION'] = df_ordenado['MODALIDAD APROBACION'].apply(APROBACION)

df_ordenado['MODALIDAD APROBACION'].value_counts(dropna=False).sort_index()

A      32
B      31
C      49
D       2
E1      2
nan     1
Name: MODALIDAD APROBACION, dtype: int64

Tipo de Obra

In [83]:
df_ordenado['TIPO OBRA'] = df_ordenado['TIPO OBRA'].astype(str)

In [84]:
df_ordenado['TIPO OBRA'].value_counts(dropna=False).sort_index()

AMPLIACION                                                         7
AMPLIACION - DEMOLICION PARCIAL                                    1
AMPLIACION - REMODELACION                                          5
AMPLIACION - REMODELACION - DEMOLICION PARCIAL                     1
AMPLIACIÓN Y REMODELACIÓN                                          2
AMPLIACIÓN/REMODELACIÓN                                            3
CERCADO - REMODELACION                                             1
CERCO PERIMETRICO                                                  3
DEMOLICION TOTAL                                                  12
DEMOLICION TOTAL                                                   6
DMEOLICION TOTAL                                                   1
LICENCIA AMP,REM,DEM.                                              1
LICENCIA AMPLIACIÓN - REMODELACIÓN                                 1
LICENCIA DE AMPLIACIÓN                                             1
LICENCIA DE AMPLIACIÓN            

In [85]:
df_ordenado['TIPO OBRA'] = df_ordenado['TIPO OBRA'].str.strip()

In [86]:
df_ordenado['TIPO OBRA'].value_counts(dropna=False).sort_index()

AMPLIACION                                                         7
AMPLIACION - DEMOLICION PARCIAL                                    1
AMPLIACION - REMODELACION                                          5
AMPLIACION - REMODELACION - DEMOLICION PARCIAL                     1
AMPLIACIÓN Y REMODELACIÓN                                          2
AMPLIACIÓN/REMODELACIÓN                                            3
CERCADO - REMODELACION                                             1
CERCO PERIMETRICO                                                  3
DEMOLICION TOTAL                                                  18
DMEOLICION TOTAL                                                   1
LICENCIA AMP,REM,DEM.                                              1
LICENCIA AMPLIACIÓN - REMODELACIÓN                                 1
LICENCIA DE AMPLIACIÓN                                             2
LICENCIA DE AMPLIACIÓN Y REMODELACIÓN                              1
LICENCIA DE AMPLIACIÓN-REMODELACIÓ

In [87]:
def OBRA(DETALLE2):
    if 'MOD. DE PROYECTO' in DETALLE2:
        return 'LICENCIA DE MODIFICACION DE PROYECTO'
    elif 'LICENCIA DE DEMOLICION' in DETALLE2:
        return 'LICENCIA DE DEMOLICION TOTAL'
    elif "LICENCIA DEMOLICION TOTAL" in DETALLE2:
        return "LICENCIA DE DEMOLICION TOTAL"
    elif "LICENCIA OBRA NUEVA - ETAPA 2" in DETALLE2:
        return "LICENCIA OBRA NUEVA ETAPA 2"
    elif "LICENCIA OBRA NUEVA - ETAPA 3" in DETALLE2:
        return "LICENCIA OBRA NUEVA ETAPA 3"
    elif "LICENCIA OBRA NUEVA - ETAPA 4" in DETALLE2:
        return "LICENCIA OBRA NUEVA ETAPA 4"
    elif "OLICION TOTAL" in DETALLE2:
        return "LICENCIA DE DEMOLICION TOTAL"
    elif "AMPLIACIÓN Y REMODELACIÓN" in DETALLE2:
        return "LICENCIA DE AMPLIACION - REMODELACION"
    elif "AMPLIACIÓN/REMODELACIÓN" in DETALLE2:
        return "LICENCIA DE AMPLIACION - REMODELACION"
    elif "AMPLIACION - REMODELACION - DEMOLICION PARCIAL" in DETALLE2:
        return "LICENCIA DE AMPLIACIÓN-REMODELACIÓN Y DEMOLICIÓN PARCIAL"
    elif "LICENCIA AMP,REM,DEM." in DETALLE2:
        return "LICENCIA DE AMPLIACIÓN-REMODELACIÓN Y DEMOLICIÓN PARCIAL"
    elif "CERCADO" in DETALLE2:
        return "LICENCIA DE CERCADO"
    elif "CERCO" in DETALLE2:
        return "LICENCIA DE CERCO"
    elif "AMPLIACION - REMODELACION" in DETALLE2:
        return "LICENCIA DE AMPLIACION - REMODELACION"
    elif "LICENCIA AMPLIACIÓN - REMODELACIÓN " in DETALLE2:
        return "LICENCIA DE AMPLIACION - REMODELACION"
    elif "REMODELACIÓN" in DETALLE2:
        return "LICENCIA DE REMODELACIÓN"
    elif "OBRA NUEVA" in DETALLE2:
        return "LICENCIA DE EDIFICACION OBRA NUEVA"
    elif "NUEVA" in DETALLE2:
        return "LICENCIA DE EDIFICACION OBRA NUEVA"
    elif "AMPLIACION - DEMOLICION PARCIAL" in DETALLE2:
        return "LICENCIA DE AMPLIACION - DEMOLICION PARCIAL"
    elif "AMPLIACION" in DETALLE2:
        return "LICENCIA DE AMPLIACIÓN"
    else:
        return DETALLE2

df_ordenado['TIPO OBRA'] = df_ordenado['TIPO OBRA'].apply(OBRA)

df_ordenado['TIPO OBRA'].value_counts(dropna=False).sort_index()

LICENCIA DE AMPLIACION - DEMOLICION PARCIAL                  1
LICENCIA DE AMPLIACION - REMODELACION                       12
LICENCIA DE AMPLIACIÓN                                       9
LICENCIA DE AMPLIACIÓN-REMODELACIÓN Y DEMOLICIÓN PARCIAL     2
LICENCIA DE CERCADO                                          1
LICENCIA DE CERCO                                            5
LICENCIA DE DEMOLICION TOTAL                                32
LICENCIA DE EDIFICACION - AMPLIACIÓN                         1
LICENCIA DE EDIFICACION OBRA NUEVA                          38
LICENCIA DE MODIFICACION DE PROYECTO                         6
LICENCIA DE REMODELACIÓN                                     5
LICENCIA OBRA NUEVA ETAPA 2                                  1
LICENCIA OBRA NUEVA ETAPA 3                                  1
LICENCIA OBRA NUEVA ETAPA 4                                  1
nan                                                          2
Name: TIPO OBRA, dtype: int64

The categories are well written.

Let's see the numeric columns. Let's identify cell that do not have numeric strings:


Numéricos

Obra Nueva

In [88]:
OBRA_NUEVA1 = df_ordenado["OBRA NUEVA"].str.contains('^\d+\.?\d*$', na=False)

In [89]:
OBRA_NUEVA1 = df_ordenado["OBRA NUEVA"].str.contains('^\d+\.?\d*$', na=False)
OBRA_NUEVA2 = OBRA_NUEVA1.fillna(False)  # O True, dependiendo de la lógica deseada

Then, we can generalize:

In [90]:
numeric_columns = ['OBRA NUEVA', 'OBRA DEMOLICIÓN',
               'OBRA REMODELACIÓN', 'OBRA AMPLIACIÓN', 'OBRA REFACCIÓN',
               'OBRA CERCO','AREA TOTAL CONSTRUIDA', 'VALOR OBRA']

In [91]:
df_ordenado[['OBRA NUEVA', 'OBRA DEMOLICIÓN',
               'OBRA REMODELACIÓN', 'OBRA AMPLIACIÓN', 'OBRA REFACCIÓN',
               'OBRA CERCO','AREA TOTAL CONSTRUIDA', 'VALOR OBRA']]=df_ordenado.iloc[:,[22,23,24,25,26,27,28,29]].replace(',','.',regex=True).replace(" ","",regex=True).replace("ML","",regex=True).replace("M2","",regex=True).replace("´","",regex=True).replace("’","",regex=True)
df_ordenado[['OBRA NUEVA', 'OBRA DEMOLICIÓN',
               'OBRA REMODELACIÓN', 'OBRA AMPLIACIÓN', 'OBRA REFACCIÓN',
               'OBRA CERCO','AREA TOTAL CONSTRUIDA', 'VALOR OBRA']]

,OBRA NUEVA,OBRA DEMOLICIÓN,OBRA REMODELACIÓN,OBRA AMPLIACIÓN,OBRA REFACCIÓN,OBRA CERCO,AREA TOTAL CONSTRUIDA,VALOR OBRA
0,-,-,97.83,208.46,-,-,208.46,225.652.12
115,-,200,-,-,-,-,-,45455.7
116,1688.79,-,-,-,-,-,1688.79,1561032.09
117,1950.08,-,-,-,-,-,1950.08,1885715.08
119,11756.55,65.58,1724.5,2714.76,-,-,-,13322075.74
...,...,...,...,...,...,...,...,...
61,469.88,-,-,-,-,-,469.88,359.246.75
63,-,194.28,-,-,-,-,-,88222.55
64,-,162.24,-,-,-,-,-,88222.55
66,-,102,-,-,-,-,-,14427.9


In [92]:
def corregir_numero(valor):
    if isinstance(valor, str):
        valor = valor.strip()
        valor = valor.replace(',', '.')
        partes = valor.split('.')
        if len(partes) > 2:
            valor_corregido = ''.join(partes[:-1]) + '.' + partes[-1]
        else:
            valor_corregido = valor
        return valor_corregido
    return valor

df_ordenado['VALOR OBRA'] = df_ordenado['VALOR OBRA'].apply(corregir_numero)

df_ordenado['VALOR OBRA'] = pd.to_numeric(df_ordenado['VALOR OBRA'], errors='coerce')

print(df_ordenado['VALOR OBRA'])

0        225652.12
115       45455.70
116     1561032.09
117     1885715.08
119    13322075.74
          ...     
61       359246.75
63        88222.55
64        88222.55
66        14427.90
179      752031.05
Name: VALOR OBRA, Length: 117, dtype: float64


In [93]:
df_ordenado[['OBRA NUEVA', 'OBRA DEMOLICIÓN',
               'OBRA REMODELACIÓN', 'OBRA AMPLIACIÓN', 'OBRA REFACCIÓN',
               'OBRA CERCO','AREA TOTAL CONSTRUIDA', 'VALOR OBRA']]

,OBRA NUEVA,OBRA DEMOLICIÓN,OBRA REMODELACIÓN,OBRA AMPLIACIÓN,OBRA REFACCIÓN,OBRA CERCO,AREA TOTAL CONSTRUIDA,VALOR OBRA
0,-,-,97.83,208.46,-,-,208.46,225652.12
115,-,200,-,-,-,-,-,45455.70
116,1688.79,-,-,-,-,-,1688.79,1561032.09
117,1950.08,-,-,-,-,-,1950.08,1885715.08
119,11756.55,65.58,1724.5,2714.76,-,-,-,13322075.74
...,...,...,...,...,...,...,...,...
61,469.88,-,-,-,-,-,469.88,359246.75
63,-,194.28,-,-,-,-,-,88222.55
64,-,162.24,-,-,-,-,-,88222.55
66,-,102,-,-,-,-,-,14427.90


In [94]:
badValues = []
for col in numeric_columns:
    currentBad = df_ordenado.loc[:, col].astype(str).str.contains('^\\d+\\.*\\d*$', na=False)
    badValues.extend(df_ordenado.loc[:, col][~currentBad])
badValues = list(set(badValues))
print(badValues)

[nan, nan, '--', '-']


We will need to replace those values with a proper missing value:

Let's keep the complete data:

In [95]:
df_ordenado.dropna(how='any',axis=0,inplace=True # keep complete
               ) # reset index
df_ordenado

,CODIGO ENTIDAD,CODIGO UBIGEO,CODIGO PAIS,DEPARTAMENTO,PROVINCIA,DISTRITO,NOMBRE UO,GOBIERNO LOCAL,RUC GOBIERNO LOCAL,NUMERO EXPEDIENTE,...,OBRA NUEVA,OBRA DEMOLICIÓN,OBRA REMODELACIÓN,OBRA AMPLIACIÓN,OBRA REFACCIÓN,OBRA CERCO,AREA TOTAL CONSTRUIDA,VALOR OBRA,FECHA CORTE,TIPO DE USO NORMALIZADO
0,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,15636-2022,...,-,-,97.83,208.46,-,-,208.46,225652.12,20221231,MULTIFAMILIAR
115,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,7494-2022,...,-,200,-,-,-,-,-,45455.70,20221231,MULTIFAMILIAR
116,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,3732-2022,...,1688.79,-,-,-,-,-,1688.79,1561032.09,20221231,MULTIFAMILIAR
117,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,3947-2022,...,1950.08,-,-,-,-,-,1950.08,1885715.08,20221231,MULTIFAMILIAR
119,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,13972-2022,...,11756.55,65.58,1724.5,2714.76,-,-,-,13322075.74,20221231,MULTIFAMILIAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,5680-2022,...,-,112,-,-,-,-,-,48224.96,20221231,UNIFAMILIAR
63,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,7501-2022,...,-,194.28,-,-,-,-,-,88222.55,20221231,UNIFAMILIAR
64,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,7507-2022,...,-,162.24,-,-,-,-,-,88222.55,20221231,UNIFAMILIAR
66,301285,150136,PE,Lima,Lima,San Miguel,Subgerencia de Obras Privadas,San Miguel,20131372184,6932-2022,...,-,102,-,-,-,-,-,14427.90,20221231,UNIFAMILIAR


## 3. Date

In [ ]:
def FECHAS(DETALLE2):
    if '1 PISO' in DETALLE2:
        return '1 PISO'
    elif '2 PISO' in DETALLE2:
        return '2 PISOS'
    elif "3 PISO" in DETALLE2:
        return "3 PISOS"
    elif "4 PISO" in DETALLE2:
        return "4 PISOS"
    elif "5 PISO" in DETALLE2:
        return "5 PISOS"
    elif "6P" in DETALLE2:
        return "6 PISOS"
    elif "6 PISO" in DETALLE2:
        return "6 PISOS"
    elif "7 PISO" in DETALLE2:
        return "7 PISOS"
    elif "7P" in DETALLE2:
        return "7 PISOS"
    elif "8 PISO" in DETALLE2:
        return "8 PISOS"
    elif "8P" in DETALLE2:
        return "8 PISOS"
    elif "9 PISO" in DETALLE2:
        return "9 PISOS"
    elif "9P" in DETALLE2:
        return "9 PISOS"
    elif "10 PISO" in DETALLE2:
        return "10 PISOS"
    elif "10P" in DETALLE2:
        return "10 PISOS"
    elif "11 PISO" in DETALLE2:
        return "11 PISOS"
    elif "12 PISO" in DETALLE2:
        return "12 PISOS"
    elif "13 PISO" in DETALLE2:
        return "13 PISOS"
    elif "14 PISO" in DETALLE2:
        return "14 PISOS"
    elif "15 PISO" in DETALLE2:
        return "15 PISOS"
    elif "16 PISO" in DETALLE2:
        return "16 PISOS"
    elif "17 PISO" in DETALLE2:
        return "17 PISOS"
    elif "17P" in DETALLE2:
        return "17 PISOS"
    elif "20 PISOS" in DETALLE2:
        return "20 PISOS"
    elif "20P" in DETALLE2:
        return "20 PISOS"
    elif "13:0" in DETALLE2:
        return ""
    else:
        return DETALLE2

df_ordenado['DETALLE ALTURA'] = df_ordenado['DETALLE ALTURA'].apply(ALTURA2)

In [98]:
df_ordenado['FECHA INICIO TRAMITE'] = pd.to_datetime(df_ordenado['FECHA INICIO TRAMITE'], format='%Y%m%d')

ValueError: time data '17//08/2021' does not match format '%Y%m%d' (match)

## 4. Save output

In [96]:
df_ordenado.to_csv('Df_ordenado_PY.csv',index=False)